In [1]:
import pandas as pd
import requests,csv
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Source: https://api.census.gov/data/2019/acs/acs5/variables.html
data_dict = {'B08133_001E':'Estimate!!Aggregate travel time to work (in minutes) AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK', 
            'B08133_007E' : 'Estimate!!Aggregate travel time to work (in minutes):!!7:00 a.m. to 7:29 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_008E' : 'Estimate!!Aggregate travel time to work (in minutes):!!7:30 a.m. to 7:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_009E' : 'Estimate!!Aggregate travel time to work (in minutes):!!8:00 a.m. to 8:29 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_010E' : 'Estimate!!Aggregate travel time to work (in minutes):!!8:30 a.m. to 8:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_011E' : 'Estimate!!Aggregate travel time to work (in minutes):!!9:00 a.m. to 9:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_012E' : 'Estimate!!Aggregate travel time to work (in minutes):!!10:00 a.m. to 10:59 a.m. AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08133_013E' : 'Estimate!!Aggregate travel time to work (in minutes):!!11:00 a.m. to 11:59 a.m.	AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_001E' : 'Estimate!!Total SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_007E' : 'Estimate!!Total:!!7:00 a.m. to 7:29 a.m.	SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_008E' : 'Estimate!!Total:!!7:30 a.m. to 7:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_009E' : 'Estimate!!Total:!!8:00 a.m. to 8:29 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_010E' : 'Estimate!!Total:!!8:30 a.m. to 8:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_011E' : 'Estimate!!Total:!!9:00 a.m. to 9:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_012E' : 'Estimate!!Total:!!10:00 a.m. to 10:59 a.m. SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08011_013E' : 'Estimate!!Total:!!11:00 a.m. to 11:59 a.m.	SEX OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK',
            'B08136_002E' : 'Estimate!!Aggregate travel time to work (in minutes):!!Car, truck, or van: AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY MEANS OF TRANSPORTATION TO WORK',
            'B08136_007E' : 'Estimate!!Aggregate travel time to work (in minutes):!!Public transportation (excluding taxicab): AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY MEANS OF TRANSPORTATION TO WORK',
            'B08006_001E' : 'Estimate!!Total SEX OF WORKERS BY MEANS OF TRANSPORTATION TO WORK',
            'B08006_002E' : 'Estimate!!Total:!!Car, truck, or van: SEX OF WORKERS BY MEANS OF TRANSPORTATION TO WORK',
            'B08006_008E' : 'Estimate!!Total:!!Public transportation (excluding taxicab): SEX OF WORKERS BY MEANS OF TRANSPORTATION TO WORK',
            'B19013_001E' : 'Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)	MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)',
            'B08201_001E' : 'Estimate!!Total: HOUSEHOLD SIZE BY VEHICLES AVAILABLE',
            'B08201_002E': 'Estimate!!Total:!!No vehicle available HOUSEHOLD SIZE BY VEHICLES AVAILABLE',
            'B08201_003E' : 'Estimate!!Total:!!1 vehicle available HOUSEHOLD SIZE BY VEHICLES AVAILABLE',
            'B08201_004E': 'Estimate!!Total:!!2 vehicles available HOUSEHOLD SIZE BY VEHICLES AVAILABLE',
            'B08201_005E': 'Estimate!!Total:!!3 vehicles available HOUSEHOLD SIZE BY VEHICLES AVAILABLE',
            'B08201_006E': 'Estimate!!Total:!!4 or more vehicles available HOUSEHOLD SIZE BY VEHICLES AVAILABLE',
            'B25003_001E': 'Estimate!!Total: TENURE',
            'B25003_002E': 'Estimate!!Total:!!Owner occupied TENURE',
            'B25003_003E': 'Estimate!!Total:!!Renter occupied TENURE',
            'B02001_001E': 'Estimate!!Total: RACE',
            'B02001_002E': 'Estimate!!Total:!!White alone RACE',
            'B02001_003E' : 'Estimate!!Total:!!Black or African American alone	RACE',
            'B02001_004E' : 'Estimate!!Total:!!American Indian and Alaska Native alone	RACE',
            'B02001_005E' : 'Estimate!!Total:!!Asian alone RACE',
            'B02001_008E' : 'Estimate!!Total:!!Two or more races: RACE',
            'B02008_001E' : 'Estimate!!Total: WHITE ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES',
            'B02009_001E': 'Estimate!!Total: BLACK OR AFRICAN AMERICAN ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES',
            'B02010_001E' : 'Estimate!!Total: AMERICAN INDIAN AND ALASKA NATIVE ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES',
            'B02011_001E' : 'Estimate!!Total: ASIAN ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES',
            'B02012_001E' : 'Estimate!!Total: NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE OR IN COMBINATION WITH ONE OR MORE OTHER RACES',
            "avg_time_agg" : "Average travel time to work, all modes of transportation = Aggregate travel time to work in minutes / Total workers",
            "avg_time_to_work_7_8am" : "Average travel time to work, all modes of transportation, for workers departing for work between 7-8am (B08133_007E + B08133_008E)/(B08011_007E+ B08011_008E)",
            "avg_time_to_work_8_9am": "Average travel time to work, all modes of transportation, for workers departing for work between 8-9am (B08133_009E + B08133_010E)/(B08011_009E+ B08011_010E)",
            "avg_time_to_work_9_10am" : "Average travel time to work, all modes of transportation, for workers departing for work between 9-10am (B08133_011E + B08133_012E)/(B08011_011E+ B08011_012E)",
            "avg_travel_ctv" : "Average travel time to work for workers travelling via car, truck, or van (B08136_002E/B08006_002E)",
            "avg_travel_pt" : "Average travel time to work for workers travelling via public transportation (B08136_007E/B08006_008E)",
            "pct_workers_ctv" : "Percentage of workers travelling to work via car, truck, or van ((df.B08006_002E/df.B08006_001E) * 100)",
            "pct_workers_pt": "Percentage of workers travelling to work via public transportation ((df.B08006_008E/df.B08006_001E) * 100)",
            "pct_households_no_car" : "Percentage of households with no car available (B08201_002E/B08201_001E) * 100)",
            "pct_households_min_car" : "Percentage of household with one car available ((B08201_001E-B08201_002E)/B08201_002E) * 100)",
            "pct_renter_occupied_units" : "Percentage of units that are renter occupied ((B25003_003E/B25003_001E) * 100)",
            "pct_owner_occupied_units" : "Percentage of units that are owner occupied (B25003_002E/B25003_001E) * 100)"}

data_dict = pd.Series(data_dict, name = "Definition")
data_dict.index.name = "col_name"
data_dict = data_dict.reset_index()

In [3]:
tables = [
    "NAME","B08133_001E","B08011_001E","B08011_007E","B08133_007E","B08011_008E","B08133_008E","B08011_009E","B08133_009E",
    "B08011_010E","B08133_010E","B08011_011E","B08133_011E","B08011_012E","B08133_012E","B08011_013E","B08133_013E",
    "B08136_002E","B08136_007E","B08136_012E","B08006_002E","B08006_008E","B08006_016E","B19013_001E","B08006_001E",
    "B08201_001E","B08201_002E","B08201_003E", "B08201_004E","B08201_005E","B08201_006E","B25003_001E","B25003_002E",
    "B25003_003E","B02001_001E","B02001_002E","B02001_003E","B02001_004E","B02001_005E","B02001_008E","B02008_001E",
    "B02009_001E","B02010_001E","B02011_001E","B02012_001E"
    ]
# ','.join(tables)

In [4]:
# years = [2010,2015,2019]
years = [2015,2019]

key = '9e840ebf4aecf1dfee3001ca5b9df8a1c5c8e541'
variables= ','.join(tables)
for y in years:
    # url=f"https://api.census.gov/data/{y}/acs/acs5?get={variables}&for=tract:*&in=state:44&key={key}"
    url=f"https://api.census.gov/data/{y}/acs/acs5?get={variables}&for=zip%20code%20tabulation%20area:*&in=state:44&key={key}"
    
    response = requests.get(url)
    data = response.json()

    with open(f"RI_ZCTA_raw_{y}.csv", "w", newline='') as writefile:
        writer = csv.writer(writefile, quoting=csv.QUOTE_ALL, delimiter=',')
        writer.writerows(data)

for y in years:
    df = pd.read_csv(f"RI_ZCTA_raw_{y}.csv")
    # B08133_001E = Estimate!!Aggregate travel time to work (in minutes) AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK
    # 
    df["avg_time_agg"] = df.B08133_001E/df.B08011_001E 
    # (Aggregate travel time (minutes) to work in minutes of workers departing between 7AM and 7:29 + Aggregate travel time (minutes) 
    # of workers departing between 7:30 and 7:59)
    
    df["avg_time_to_work_7_8am"] = (df.B08133_007E + df.B08133_008E)/(df.B08011_007E+ df.B08011_008E)
    df["avg_time_to_work_8_9am"] = (df.B08133_009E + df.B08133_010E)/(df.B08011_009E+ df.B08011_010E)
    df["avg_time_to_work_9_10am"] = (df.B08133_011E + df.B08133_012E)/(df.B08011_011E+ df.B08011_012E)

    df["avg_travel_ctv"] = df.B08136_002E/df.B08006_002E # Average travel time to work by car, truck, or van
    df['avg_travel_pt'] =  df.B08136_007E/df.B08006_008E # Average travel time to work by public transit, excluding taxi

    df['pct_workers_ctv'] = (df.B08006_002E/df.B08006_001E) * 100
    df['pct_workers_pt'] = (df.B08006_008E/df.B08006_001E) * 100

    df['pct_households_no_car'] = (df.B08201_002E/df.B08201_001E) * 100
    df['pct_households_one_car'] = (df.B08201_003E/df.B08201_001E) * 100
    df['pct_households_two_cars'] = (df.B08201_004E/df.B08201_001E) * 100
    df['pct_households_three_cars'] = (df.B08201_005E/df.B08201_001E) * 100
    df['pct_households_four_more_cars'] = (df.B08201_006E/df.B08201_001E) * 100

    df['pct_households_min_car'] = ((df.B08201_001E-df.B08201_002E)/df.B08201_001E) * 100

    df['pct_renter_occupied_units'] = (df.B25003_003E/df.B25003_001E) * 100
    df['pct_owner_occupied_units'] = (df.B25003_002E/df.B25003_001E) * 100

    df = df.rename(columns = {'B19013_001E':'mhi', 'B02001_001E':'total_population_race','B02001_002E':'num_white_alone','B02001_003E':'num_baa_alone'})

    df['pct_white'] = (df.num_white_alone/df.total_population_race) * 100
    df['pct_nonwhite'] = ((df.total_population_race - df.num_white_alone)/df.total_population_race) * 100

    df['year'] = y
    df.to_csv(f"RI_ZCTA_mod_{y}.csv")
    # data_2015['avg_travel_other'] = data_2015.B08136_012E/data_2015.B08006_016E

# data_2010 = pd.read_csv('RI_ZCTA_mod_2010.csv')
data_2015 = pd.read_csv("RI_ZCTA_mod_2015.csv")
data_2019 = pd.read_csv("RI_ZCTA_mod_2019.csv")

# creating zip column to match zip on shapefile
# data_2010[['NAME','zip']] = data_2010.NAME.str.split(' ', expand=True)
data_2015[['NAME','zip']] = data_2015.NAME.str.split(' ', expand=True)
data_2019[['NAME','zip']] = data_2019.NAME.str.split(' ', expand=True)

with pd.ExcelWriter('ZCTA_data.xlsx') as writer:
    data_dict.to_excel(writer, sheet_name="Data Dictionary")
    # data_2010.to_excel(writer, sheet_name="ACS_2010")
    data_2015.to_excel(writer, sheet_name="ACS_2015")
    data_2019.to_excel(writer, sheet_name="ACS_2019")

In [5]:
all_years = pd.concat([data_2015,data_2019])
# all_years = pd.concat([data_2010,data_2015,data_2019])

all_years.to_csv('all_year_data.csv',index=False)

In [12]:
tract_tables = [
    "NAME","B08133_001E","B08011_001E","B08011_007E","B08133_007E","B08011_008E","B08133_008E","B08011_009E","B08133_009E",
    "B08011_010E","B08133_010E","B08011_011E","B08133_011E","B08011_012E","B08133_012E","B08011_013E","B08133_013E",
    "B08006_002E","B08006_008E","B08006_016E","B19013_001E","B08006_001E",
    "B08201_001E","B08201_002E","B08201_003E", "B08201_004E","B08201_005E","B08201_006E","B25003_001E","B25003_002E",
    "B25003_003E","B02001_001E","B02001_002E","B02001_003E","B02001_004E","B02001_005E","B02001_008E","B02008_001E",
    "B02009_001E","B02010_001E","B02011_001E","B02012_001E"
    ]

In [14]:
# years = [2010,2015,2019]
years = [2010,2015,2019]

key = '9e840ebf4aecf1dfee3001ca5b9df8a1c5c8e541'
variables= ','.join(tract_tables)
for y in years:
    # url=f"https://api.census.gov/data/{y}/acs/acs5?get={variables}&for=tract:*&in=state:44&key={key}"
    url=f'https://api.census.gov/data/{y}/acs/acs5?get={variables}&for=tract:*&in=state:44&in=county:007&key={key}'
    
    response = requests.get(url)
    data = response.json()

    with open(f"RI_tract_raw_{y}.csv", "w", newline='') as writefile:
        writer = csv.writer(writefile, quoting=csv.QUOTE_ALL, delimiter=',')
        writer.writerows(data)

for y in years:
    df = pd.read_csv(f"RI_tract_raw_{y}.csv")
    # B08133_001E = Estimate!!Aggregate travel time to work (in minutes) AGGREGATE TRAVEL TIME TO WORK (IN MINUTES) OF WORKERS BY TIME OF DEPARTURE TO GO TO WORK
    # 
    df["avg_time_agg"] = df.B08133_001E/df.B08011_001E 
    # (Aggregate travel time (minutes) to work in minutes of workers departing between 7AM and 7:29 + Aggregate travel time (minutes) 
    # of workers departing between 7:30 and 7:59)
    
    df["avg_time_to_work_7_8am"] = (df.B08133_007E + df.B08133_008E)/(df.B08011_007E+ df.B08011_008E)
    df["avg_time_to_work_8_9am"] = (df.B08133_009E + df.B08133_010E)/(df.B08011_009E+ df.B08011_010E)
    df["avg_time_to_work_9_10am"] = (df.B08133_011E + df.B08133_012E)/(df.B08011_011E+ df.B08011_012E)

    # df["avg_travel_ctv"] = df.B08136_002E/df.B08006_002E # Average travel time to work by car, truck, or van
    # df['avg_travel_pt'] =  df.B08136_007E/df.B08006_008E # Average travel time to work by public transit, excluding taxi

    df['pct_workers_ctv'] = (df.B08006_002E/df.B08006_001E) * 100
    df['pct_workers_pt'] = (df.B08006_008E/df.B08006_001E) * 100

    df['pct_households_no_car'] = (df.B08201_002E/df.B08201_001E) * 100
    df['pct_households_min_car'] = ((df.B08201_001E-df.B08201_002E)/df.B08201_002E) * 100

    df['pct_renter_occupied_units'] = (df.B25003_003E/df.B25003_001E) * 100
    df['pct_owner_occupied_units'] = (df.B25003_002E/df.B25003_001E) * 100

    df = df.rename(columns = {'B19013_001E':'mhi', 'B02001_001E':'total_population_race','B02001_002E':'num_white_alone','B02001_003E':'num_baa_alone'})

    df['pct_white'] = (df.num_white_alone/df.total_population_race) * 100
    df['pct_nonwhite'] = ((df.total_population_race - df.num_white_alone)/df.total_population_race) * 100

    df['year'] = y
    df.to_csv(f"RI_tract_mod_{y}.csv")
    # data_2015['avg_travel_other'] = data_2015.B08136_012E/data_2015.B08006_016E

data_2010 = pd.read_csv('RI_tract_mod_2010.csv')
data_2015 = pd.read_csv("RI_tract_mod_2015.csv")
data_2019 = pd.read_csv("RI_tract_mod_2019.csv")

# creating zip column to match zip on shapefile
# data_2010[['NAME','zip']] = data_2010.NAME.str.split(' ', expand=True)
# data_2015[['NAME','zip']] = data_2015.NAME.str.split(' ', expand=True)
# data_2019[['NAME','zip']] = data_2019.NAME.str.split(' ', expand=True)

with pd.ExcelWriter('tract_data.xlsx') as writer:
    data_dict.to_excel(writer, sheet_name="Data Dictionary")
    data_2010.to_excel(writer, sheet_name="ACS_2010")
    data_2015.to_excel(writer, sheet_name="ACS_2015")
    data_2019.to_excel(writer, sheet_name="ACS_2019")

In [ ]:
# all_years = pd.concat([data_2015,data_2019])
all_years = pd.concat([data_2010,data_2015,data_2019])

all_years.to_csv('tract_all_year_data.csv',index=False)